In [1]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from datetime import datetime
from dateutil.parser import parse
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error,
    precision_score, recall_score, f1_score, 
    r2_score, confusion_matrix
)
from scipy.stats import variation
import warnings
from gluonts.evaluation.metrics import quantile_loss
warnings.filterwarnings('ignore')

@dataclass
class DRPSeriesState:
    """单条时间序列的状态跟踪"""
    last_demand: float = 0
    last_interval: float = 1
    demand_var: float = 0
    forecast: float = 0
    demand_alpha: float = 0.1
    interval_alpha: float = 0.1
    lead_time: int = 1
    safety_factor: float = 1.5
    last_non_zero_idx: int = -1
    demand_history: List[float] = None
    interval_history: List[float] = None

class DRPModel:
    """完整的DRP模型实现，包含所有指标计算"""
    
    def __init__(self, 
                 lead_time: int = 1,
                 safety_factor: float = 1.5,
                 demand_alpha: float = 0.1,
                 interval_alpha: float = 0.1):
        self.params = {
            'lead_time': lead_time,
            'safety_factor': safety_factor,
            'demand_alpha': demand_alpha,
            'interval_alpha': interval_alpha
        }
        self.series_states = {}
    
    def _safe_parse_date(self, date_str: str) -> Optional[datetime]:
        """安全解析日期，处理无效日期"""
        try:
            return parse(str(date_str))
        except (ValueError, TypeError):
            # 尝试处理常见的格式问题
            if str(date_str).count('-') == 2:
                parts = str(date_str).split('-')
                try:
                    # 尝试将日期调整为有效日期（如将2月30日改为28日）
                    fixed_date = f"{parts[0]}-{parts[1]}-28"
                    return parse(fixed_date)
                except:
                    return None
            return None
    
    def fit(self, df: pd.DataFrame) -> Dict[str, DRPSeriesState]:
        """拟合模型到训练数据"""
        # 处理日期列
        date_strs = df.columns.astype(str).tolist()
        valid_dates = []
        date_indices = []
        
        for i, d in enumerate(date_strs):
            dt = self._safe_parse_date(d)
            if dt is not None:
                valid_dates.append(dt)
                date_indices.append(i)
        
        if not valid_dates:
            raise ValueError("没有有效的日期列，请检查数据格式")
        
        for part_id, series in df.iterrows():
            state = DRPSeriesState(
                **self.params,
                demand_history=[],
                interval_history=[]
            )
            
            # 获取非零需求点
            non_zero_mask = series > 0
            non_zero_values = series[non_zero_mask].values
            non_zero_dates = [valid_dates[i] for i in np.where(non_zero_mask)[0] if i in date_indices]
            
            if len(non_zero_values) < 2:
                self.series_states[part_id] = state
                continue
                
            # 初始化状态
            state.last_demand = non_zero_values[0]
            state.demand_history.append(state.last_demand)
            
            # 计算间隔
            intervals = []
            for i in range(1, len(non_zero_dates)):
                delta = (non_zero_dates[i] - non_zero_dates[i-1]).days
                intervals.append(delta)
            
            if intervals:
                state.interval_history = intervals
                state.last_interval = np.mean(intervals)
            
            # 计算方差
            if len(non_zero_values) > 1:
                state.demand_var = np.var(non_zero_values)
            
            self.series_states[part_id] = state
        
        return self.series_states
    
    def predict(self, part_ids: List[str], horizon: int = 1) -> Dict[str, np.ndarray]:
        """为多个时间序列生成预测"""
        forecasts = {}
        for part_id in part_ids:
            if part_id not in self.series_states:
                forecasts[part_id] = np.zeros(horizon)
                continue
                
            state = self.series_states[part_id]
            forecast_value = state.last_demand / state.last_interval if state.last_interval > 0 else 0
            forecasts[part_id] = np.full(horizon, forecast_value)
        
        return forecasts
    
    def get_inventory_plans(self, part_ids: List[str]) -> Dict[str, Dict]:
        """获取库存计划参数"""
        plans = {}
        for part_id in part_ids:
            if part_id not in self.series_states:
                plans[part_id] = None
                continue
                
            state = self.series_states[part_id]
            lead_time_demand = state.last_demand * state.lead_time
            safety_stock = state.safety_factor * np.sqrt(max(0, state.demand_var)) * np.sqrt(state.lead_time)
            
            plans[part_id] = {
                'reorder_point': max(0, lead_time_demand + safety_stock),
                'safety_stock': safety_stock,
                'lead_time_demand': lead_time_demand,
                'demand_variance': state.demand_var,
                'forecast': state.last_demand / state.last_interval if state.last_interval > 0 else 0
            }
        
        return plans

class DRPEvaluator:
    """完整的评估器，包含所有指标"""
    
    @staticmethod
    def mean_absolute_scaled_error(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray) -> float:
        scaling_factor = np.mean(np.abs(np.diff(y_train)))
        return np.mean(np.abs(y_true - y_pred)) / scaling_factor if scaling_factor > 0 else np.nan
    
    @staticmethod
    def root_mean_square_scaled_error(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray) -> float:
        scaling_factor = np.mean(np.square(np.diff(y_train)))
        return np.sqrt(np.mean(np.square(y_true - y_pred)) / scaling_factor) if scaling_factor > 0 else np.nan
    
    @staticmethod
    def quantile_loss(y_true: np.ndarray, y_pred: np.ndarray, q: float) -> float:
        errors = y_true - y_pred
        return np.mean(np.maximum(q * errors, (q - 1) * errors))
    
    @staticmethod
    def bias_amount(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        return np.sum(y_pred) - np.sum(y_true)
    
    @staticmethod
    def rmsse(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray, h: int = 1) -> float:
        numerator = np.sum(np.square(y_true - y_pred))
        denominator = np.mean(np.square(np.diff(y_train)))
        return np.sqrt(numerator / (denominator * h)) if denominator > 0 else np.nan
    
    @staticmethod
    def evaluate(
        y_true: Dict[str, np.ndarray],
        y_pred: Dict[str, np.ndarray],
        y_train: Dict[str, np.ndarray],
        test_month_num: int = 1
    ) -> Dict[str, float]:
        """计算全部指标"""
        # 合并所有序列
        true_all = np.concatenate(list(y_true.values()))
        pred_all = np.concatenate(list(y_pred.values()))
        
        # 基础指标
        metrics = {
            'R2': r2_score(true_all, pred_all),
            'RMSE': mean_squared_error(true_all, pred_all, squared=False),
            'MAE': mean_absolute_error(true_all, pred_all),
            'Bias': DRPEvaluator.bias_amount(true_all, pred_all)
        }
        
        # 非零指标
        non_zero_mask = true_all > 0
        if np.any(non_zero_mask):
            metrics.update({
                'RMSE_non_zero': mean_squared_error(true_all[non_zero_mask], pred_all[non_zero_mask], squared=False),
                'MAE_non_zero': mean_absolute_error(true_all[non_zero_mask], pred_all[non_zero_mask])
            })
        
        # 零值指标
        zero_mask = true_all == 0
        if np.any(zero_mask):
            zero_pred = pred_all == 0
            cm = confusion_matrix(zero_mask, zero_pred)
            metrics.update({
                'Precision_zero': precision_score(zero_mask, zero_pred, zero_division=0),
                'Recall_zero': recall_score(zero_mask, zero_pred, zero_division=0),
                'F1_zero': f1_score(zero_mask, zero_pred, zero_division=0),
                'Confusion_Matrix': cm
            })
        
        # 分位数损失
        ##QuantileLoss指标
        testmonth_num = 1
        LS_list = [(0,1)]  #L表示相对于第1个预测时间t0的QuantileLoss区间起始点，S表示QuantileLoss区间长度
        for LS_pair in LS_list:
            L = LS_pair[0]
            S = LS_pair[1]
            QuantileLoss2 = 0.0
            test_y2 = 0.0
            num = int(len(pred_all) / testmonth_num)  ##时间序列sku数量
            for i in range(num):
                QuantileLoss2 += quantile_loss(true_all[(testmonth_num * i + L):(testmonth_num * i + L + S)],
                                              pred_all[(testmonth_num * i + L):(testmonth_num * i + L + S)],
                                              q=0.5)
                test_y2 += np.sum(true_all[(testmonth_num * i + L):(testmonth_num * i + L + S)])
            #print(QuantileLoss2,test_y2)
            QuantileLoss2 = QuantileLoss2 / test_y2
            print('L=%d,S=%d,rou=50%%时的QuantileLoss值：%f' % (L,S,QuantileLoss2))
            metrics[f'QuantileLoss_50'] = QuantileLoss2

        '''
        if np.sum(true_all) > 0:
            for q in [0.1, 0.5, 0.9]:
                metrics[f'QuantileLoss_{int(q*100)}'] = DRPEvaluator.quantile_loss(true_all, pred_all, q)
        '''
        # 分段评估
        segments = {
            '0': (0, 0),
            '0_50': (0.1, 50),
            '50_100': (50, 100),
            '100+': (100, np.inf)
        }
        
        for seg_name, (low, high) in segments.items():
            if low == 0 and high == 0:
                mask = true_all == 0
            else:
                mask = (true_all > low) & (true_all <= high)
            
            if np.sum(mask) > 0:
                metrics.update({
                    f'RMSE_{seg_name}': mean_squared_error(true_all[mask], pred_all[mask], squared=False),
                    f'MAE_{seg_name}': mean_absolute_error(true_all[mask], pred_all[mask])
                })
        
        # 计算MASE和RMSSE（按序列计算后取平均）
        mase_values = []
        rmsse_values = []
        mase_nz_values = []
        rmsse_nz_values = []
        
        for part_id in y_true:
            y_t = y_true[part_id]
            y_p = y_pred[part_id]
            y_tr = y_train[part_id]
            
            # 总体指标
            mase = DRPEvaluator.mean_absolute_scaled_error(y_t, y_p, y_tr)
            rmsse = DRPEvaluator.rmsse(y_t, y_p, y_tr, test_month_num)
            if not np.isnan(mase):
                mase_values.append(mase)
            if not np.isnan(rmsse):
                rmsse_values.append(rmsse)
            
            # 非零指标
            nz_mask = y_t > 0
            if np.any(nz_mask):
                mase_nz = DRPEvaluator.mean_absolute_scaled_error(y_t[nz_mask], y_p[nz_mask], y_tr)
                rmsse_nz = DRPEvaluator.rmsse(y_t[nz_mask], y_p[nz_mask], y_tr, test_month_num)
                if not np.isnan(mase_nz):
                    mase_nz_values.append(mase_nz)
                if not np.isnan(rmsse_nz):
                    rmsse_nz_values.append(rmsse_nz)
        
        metrics.update({
            'MASE': np.mean(mase_values) if mase_values else np.nan,
            'RMSSE': np.mean(rmsse_values) if rmsse_values else np.nan,
            'MASE_non_zero': np.mean(mase_nz_values) if mase_nz_values else np.nan,
            'RMSSE_non_zero': np.mean(rmsse_nz_values) if rmsse_nz_values else np.nan
        })
        
        return metrics

def time_series_cross_validate(
    df: pd.DataFrame,
    model_class,
    n_splits: int = 5,
    horizon: int = 12,
    **model_params
) -> Dict[str, List[float]]:
    """时间序列交叉验证"""
    all_metrics = []
    n_periods = df.shape[1]
    test_size = horizon
    train_size = n_periods - test_size * n_splits
    
    if train_size <= 0:
        raise ValueError("交叉验证设置不合理：n_splits * horizon 大于总时间长度")
    
    for i in range(n_splits):
        train_end = train_size + i * test_size
        test_end = min(train_end + test_size, n_periods)
        
        train_df = df.iloc[:, :train_end]
        test_df = df.iloc[:, train_end:test_end]
        
        # 训练模型
        model = model_class(**model_params)
        model.fit(train_df)
        
        # 预测
        preds = model.predict(test_df.index.tolist(), horizon=test_size)
        
        # 准备评估数据
        y_true = {k: v.values for k, v in test_df.iterrows()}
        y_train = {k: v.values for k, v in train_df.iterrows()}
        
        # 评估
        metrics = DRPEvaluator.evaluate(y_true, preds, y_train, test_month_num=horizon)
        all_metrics.append(metrics)
    
    # 聚合结果
    final_metrics = {}
    for metric in all_metrics[0].keys():
        if metric == 'Confusion_Matrix':
            final_metrics[metric] = sum([m[metric] for m in all_metrics])
        else:
            values = [m[metric] for m in all_metrics if not np.isnan(m[metric])]
            final_metrics[metric] = np.mean(values) if values else np.nan
    
    return final_metrics

# 使用示例
if __name__ == "__main__":
    # 加载数据
    # 加载数据(确保第一列是Part ID，后面是日期列)
    #data = pd.read_csv('E:\ZIP-DeepAR代码\data\InterSim层次聚类后的Q料202001-202302(halfmonth).csv', index_col=0)
    #data = pd.read_csv('E:\ZIP-DeepAR代码\data\salestv_data.csv', index_col=0)
    data = pd.read_csv('E:\ZIP-DeepAR代码\data\carpartsdelete70.csv', index_col=0)
        #将列名转化为202001 ~ 202604月份，避免freq=15d的2-30日期问题
    import calendar
    from datetime import datetime, timedelta

    date_str = '1998-01'
    date_format = '%Y-%m'
    num_months = 51 #51 76 1913

    dates = []
    current_date = datetime.strptime(date_str, date_format)

    for i in range(num_months):
        dates.append(current_date.strftime(date_format))
        num_days = calendar.monthrange(current_date.year, current_date.month)[1]
        current_date += timedelta(days=num_days)

    data.columns = dates
    
    
    # 检查日期列
    print("前5个日期列:", data.columns[:5])
    
    # 运行交叉验证
    cv_results = time_series_cross_validate(
        data,
        DRPModel,
        n_splits=3,
        horizon=6,
        lead_time=2,
        safety_factor=1.5,
        demand_alpha=0.1,
        interval_alpha=0.1
    )
    
    print("\n交叉验证结果:")
    for metric, value in cv_results.items():
        if metric != 'Confusion_Matrix':
            print(f"{metric}: {value:.4f}")
    print("混淆矩阵:\n", cv_results['Confusion_Matrix'])
    
    # 完整训练和评估
    train_size = 47 #47 72 1909
    train_df = data.iloc[:, :train_size]
    test_df = data.iloc[:, train_size:]
    
    model = DRPModel(
        lead_time=2,
        safety_factor=1.5,
        demand_alpha=0.1,
        interval_alpha=0.1
    )
    model.fit(train_df)
    
    # 预测
    preds = model.predict(test_df.index.tolist(), horizon=test_df.shape[1])
    
    # 评估
    y_true = {k: v.values for k, v in test_df.iterrows()}
    y_train = {k: v.values for k, v in train_df.iterrows()}
    metrics = DRPEvaluator.evaluate(y_true, preds, y_train)
    
    print("\n测试集评估结果:")
    for metric, value in metrics.items():
        if metric != 'Confusion_Matrix':
            print(f"{metric}: {value:.4f}")
    print("混淆矩阵:\n", metrics['Confusion_Matrix'])
    
    # 查看库存计划
    inventory_plans = model.get_inventory_plans(test_df.index.tolist()[:3])
    print("\n示例库存计划:")
    for part_id, plan in inventory_plans.items():
        print(f"{part_id}:")
        for k, v in plan.items():
            print(f"  {k}: {v:.2f}")


前5个日期列: Index(['1998-01', '1998-02', '1998-03', '1998-04', '1998-05'], dtype='object')
L=0,S=1,rou=50%时的QuantileLoss值：1.010971
L=0,S=1,rou=50%时的QuantileLoss值：1.014479
L=0,S=1,rou=50%时的QuantileLoss值：1.023553

交叉验证结果:
R2: -0.2901
RMSE: 1.2685
MAE: 0.6550
Bias: -3089.4598
RMSE_non_zero: 2.0639
MAE_non_zero: 1.6605
Precision_zero: 0.0000
Recall_zero: 0.0000
F1_zero: 0.0000
QuantileLoss_50: 1.0163
RMSE_0: 0.0569
MAE_0: 0.0436
RMSE_0_50: 2.0639
MAE_0_50: 1.6605
MASE: 0.7229
RMSSE: 0.6598
MASE_non_zero: 1.6811
RMSSE_non_zero: 0.7401
混淆矩阵:
 [[5833    0]
 [9593    0]]
L=0,S=1,rou=50%时的QuantileLoss值：1.024572

测试集评估结果:
R2: -0.2264
RMSE: 1.1937
MAE: 0.5682
Bias: -1758.3957
RMSE_non_zero: 2.0869
MAE_non_zero: 1.6545
Precision_zero: 0.0000
Recall_zero: 0.0000
F1_zero: 0.0000
QuantileLoss_50: 1.0246
RMSE_0: 0.0525
MAE_0: 0.0410
RMSE_0_50: 2.0869
MAE_0_50: 1.6545
MASE: 0.5944
RMSSE: 1.0536
MASE_non_zero: 1.5987
RMSSE_non_zero: 1.4563
混淆矩阵:
 [[1120    0]
 [2308    0]]

示例库存计划:
21053055:
  reorder_point

In [7]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from datetime import datetime
from dateutil.parser import parse
from scipy.stats import variation
import warnings
from gluonts.evaluation.metrics import quantile_loss
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error,
    precision_score, recall_score, f1_score, 
    r2_score, confusion_matrix
)
warnings.filterwarnings('ignore')

@dataclass
class DRPResult:
    """Result container similar to CrostonResult"""
    forecast: np.ndarray
    demand_series: np.ndarray
    interval_series: np.ndarray
    alpha: float
    initial_demand: float
    initial_interval: float

class DRPModel:
    """DRP model implementation aligned with Croston's interface"""
    
    def __init__(self, 
                 lead_time: int = 1,
                 safety_factor: float = 1.5,
                 alpha: float = 0.1):
        self.lead_time = lead_time
        self.safety_factor = safety_factor
        self.alpha = alpha
        self.fitted_models = []
    
    def _safe_parse_date(self, date_str: str) -> Optional[datetime]:
        """Safely parse dates, handling invalid dates"""
        try:
            return parse(str(date_str))
        except (ValueError, TypeError):
            if str(date_str).count('-') == 2:
                parts = str(date_str).split('-')
                try:
                    fixed_date = f"{parts[0]}-{parts[1]}-28"
                    return parse(fixed_date)
                except:
                    return None
            return None
    
    def fit(self, data: pd.DataFrame) -> List[DRPResult]:
        """Fit model to training data, returning DRPResult objects"""
        date_strs = data.columns.astype(str).tolist()
        valid_dates = []
        date_indices = []
        
        for i, d in enumerate(date_strs):
            dt = self._safe_parse_date(d)
            if dt is not None:
                valid_dates.append(dt)
                date_indices.append(i)
        
        if not valid_dates:
            raise ValueError("No valid date columns found")
        
        results = []
        for series in data.values:
            non_zero_indices = np.where(series > 0)[0]
            valid_non_zero_indices = [i for i in non_zero_indices if i in date_indices]
            
            if len(valid_non_zero_indices) == 0:
                results.append(DRPResult(
                    forecast=np.zeros(len(series)),
                    demand_series=np.zeros(len(series)),
                    interval_series=np.zeros(len(series)),
                    alpha=self.alpha,
                    initial_demand=0,
                    initial_interval=0
                ))
                continue
                
            # Extract demand and intervals
            demand = series[valid_non_zero_indices]
            intervals = np.diff(valid_non_zero_indices, prepend=-1)
            intervals[0] = valid_non_zero_indices[0] + 1  # Handle first interval
            
            # Initialize series
            demand_series = np.zeros(len(series))
            interval_series = np.zeros(len(series))
            
            # Initial values
            initial_demand = demand[0]
            initial_interval = intervals[0]
            
            # Update series
            current_demand = initial_demand
            current_interval = initial_interval
            last_non_zero_idx = valid_non_zero_indices[0]
            
            for i in range(len(series)):
                if series[i] > 0 and i in date_indices:
                    current_demand = self.alpha * series[i] + (1 - self.alpha) * current_demand
                    if i > 0:
                        prev_non_zero = [x for x in valid_non_zero_indices if x < i]
                        if prev_non_zero:
                            interval = i - prev_non_zero[-1]
                        else:
                            interval = i + 1
                        current_interval = self.alpha * interval + (1 - self.alpha) * current_interval
                        last_non_zero_idx = i
                
                demand_series[i] = current_demand
                interval_series[i] = current_interval
            
            # Calculate forecasts
            forecast = np.where(interval_series > 0, demand_series / interval_series, 0)
            
            results.append(DRPResult(
                forecast=forecast,
                demand_series=demand_series,
                interval_series=interval_series,
                alpha=self.alpha,
                initial_demand=initial_demand,
                initial_interval=initial_interval
            ))
        
        self.fitted_models = results
        return results
    
    def predict(self, steps: int = 1) -> np.ndarray:
        """Predict future values"""
        forecasts = []
        for model in self.fitted_models:
            last_demand = model.demand_series[-1]
            last_interval = model.interval_series[-1]
            
            if last_interval <= 0:
                forecasts.append(np.zeros(steps))
            else:
                forecasts.append(np.full(steps, last_demand / last_interval))
        
        return np.array(forecasts)
    
    def get_inventory_plans(self) -> List[Dict]:
        """Get inventory plans for all series"""
        plans = []
        for model in self.fitted_models:
            demand_var = np.var(model.demand_series[model.demand_series > 0]) if np.any(model.demand_series > 0) else 0
            lead_time_demand = model.demand_series[-1] * self.lead_time
            safety_stock = self.safety_factor * np.sqrt(max(0, demand_var)) * np.sqrt(self.lead_time)
            
            plans.append({
                'reorder_point': max(0, lead_time_demand + safety_stock),
                'safety_stock': safety_stock,
                'lead_time_demand': lead_time_demand,
                'demand_variance': demand_var,
                'forecast': model.demand_series[-1] / model.interval_series[-1] if model.interval_series[-1] > 0 else 0
            })
        
        return plans


class DRPEvaluator:
    """Evaluator using same metrics as CrostonEvaluator"""
    
    @staticmethod
    def mean_absolute_scaled_error(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray) -> float:
        scaling_factor = np.mean(np.abs(np.diff(y_train)))
        return np.mean(np.abs(y_true - y_pred)) / scaling_factor if scaling_factor > 0 else np.nan
    
    @staticmethod
    def root_mean_square_scaled_error(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray) -> float:
        scaling_factor = np.mean(np.square(np.diff(y_train)))
        return np.sqrt(np.mean(np.square(y_true - y_pred)) / scaling_factor) if scaling_factor > 0 else np.nan
    
    @staticmethod
    def quantile_loss(y_true: np.ndarray, y_pred: np.ndarray, q: float) -> float:
        errors = y_true - y_pred
        return np.mean(np.maximum(q * errors, (q - 1) * errors))
    
    @staticmethod
    def evaluate(
        y_true: np.ndarray,
        y_pred: np.ndarray,
        y_train: np.ndarray,
        test_month_num: int = 1
    ) -> dict:
        metrics = {}
        y_true_flat = y_true.flatten()
        y_pred_flat = y_pred.flatten()
        
        # Non-zero metrics
        non_zero_mask = y_true_flat > 0
        y_true_non_zero = y_true_flat[non_zero_mask]
        y_pred_non_zero = y_pred_flat[non_zero_mask]
        
        # R2 score
        metrics['R2'] = r2_score(y_true_flat, y_pred_flat)
        
        # RMSE
        metrics['RMSE'] = mean_squared_error(y_true_flat, y_pred_flat, squared=False)
        metrics['RMSE_non_zero'] = mean_squared_error(y_true_non_zero, y_pred_non_zero, squared=False) if len(y_true_non_zero) > 0 else np.nan
        
        # MAE
        metrics['MAE'] = mean_absolute_error(y_true_flat, y_pred_flat)
        metrics['MAE_non_zero'] = mean_absolute_error(y_true_non_zero, y_pred_non_zero) if len(y_true_non_zero) > 0 else np.nan
        
        # MASE
        mase_values = []
        mase_non_zero_values = []
        for i in range(len(y_true)):
            mase = DRPEvaluator.mean_absolute_scaled_error(y_true[i], y_pred[i], y_train[i])
            mase_values.append(mase)
            
            # Non-zero MASE
            non_zero_mask = y_true[i] > 0
            if np.any(non_zero_mask):
                mase_nz = DRPEvaluator.mean_absolute_scaled_error(
                    y_true[i][non_zero_mask], 
                    y_pred[i][non_zero_mask], 
                    y_train[i]
                )
                mase_non_zero_values.append(mase_nz)
        
        metrics['MASE'] = np.nanmean(mase_values)
        metrics['MASE_non_zero'] = np.nanmean(mase_non_zero_values) if mase_non_zero_values else np.nan
        
        # RMSSE
        rmsse_values = []
        rmsse_non_zero_values = []
        for i in range(len(y_true)):
            rmsse = DRPEvaluator.root_mean_square_scaled_error(y_true[i], y_pred[i], y_train[i])
            rmsse_values.append(rmsse)
            
            # Non-zero RMSSE
            non_zero_mask = y_true[i] > 0
            if np.any(non_zero_mask):
                rmsse_nz = DRPEvaluator.root_mean_square_scaled_error(
                    y_true[i][non_zero_mask], 
                    y_pred[i][non_zero_mask], 
                    y_train[i]
                )
                rmsse_non_zero_values.append(rmsse_nz)
        
        metrics['RMSSE'] = np.nanmean(rmsse_values)
        metrics['RMSSE_non_zero'] = np.nanmean(rmsse_non_zero_values) if rmsse_non_zero_values else np.nan
        
        # Quantile Loss
        testmonth_num = 1
        LS_list = [(0,1)]
        for LS_pair in LS_list:
            L = LS_pair[0]
            S = LS_pair[1]
            QuantileLoss2 = 0.0
            test_y2 = 0.0
            num = int(len(y_pred_flat) / testmonth_num)
            for i in range(num):
                QuantileLoss2 += quantile_loss(
                    y_true_flat[(testmonth_num * i + L):(testmonth_num * i + L + S)],
                    y_pred_flat[(testmonth_num * i + L):(testmonth_num * i + L + S)],
                    q=0.5
                )
                test_y2 += np.sum(y_true_flat[(testmonth_num * i + L):(testmonth_num * i + L + S)])
            QuantileLoss2 = QuantileLoss2 / test_y2
            metrics[f'QuantileLoss_50'] = QuantileLoss2
        
        # Zero metrics
        test_y_01 = [1 if x == 0 else 0 for x in y_true_flat]
        test_predict_01 = [1 if x == 0 else 0 for x in y_pred_flat]
        metrics['Precision'] = precision_score(test_y_01, test_predict_01)
        metrics['Recall'] = recall_score(test_y_01, test_predict_01)
        metrics['F1'] = f1_score(test_y_01, test_predict_01)
        
        # Segment metrics
        segments = {
            '0': (0, 0),
            '0_50': (0, 50),
            '50_100': (50, 100),
            '100_above': (100, np.inf)
        }
        
        for name, (lower, upper) in segments.items():
            if lower == upper:
                mask = y_true_flat == lower
            elif upper == np.inf:
                mask = y_true_flat > lower
            else:
                mask = (y_true_flat > lower) & (y_true_flat <= upper)
                
            y_true_seg = y_true_flat[mask]
            y_pred_seg = y_pred_flat[mask]
            
            if len(y_true_seg) > 0:
                metrics[f'MAE_{name}'] = mean_absolute_error(y_true_seg, y_pred_seg)
                metrics[f'RMSE_{name}'] = mean_squared_error(y_true_seg, y_pred_seg, squared=False)
            else:
                metrics[f'MAE_{name}'] = np.nan
                metrics[f'RMSE_{name}'] = np.nan
        
        return metrics

def time_series_cross_validate(
    df: pd.DataFrame,
    model_class,
    n_splits: int = 5,
    horizon: int = 12,
    **model_params
) -> Dict[str, List[float]]:
    """Time series cross validation"""
    all_metrics = []
    n_periods = df.shape[1]
    test_size = horizon
    train_size = n_periods - test_size * n_splits
    
    if train_size <= 0:
        raise ValueError("Invalid CV setup: n_splits * horizon > total periods")
    
    for i in range(n_splits):
        train_end = train_size + i * test_size
        test_end = min(train_end + test_size, n_periods)
        
        train_df = df.iloc[:, :train_end]
        test_df = df.iloc[:, train_end:test_end]
        
        # Train model
        model = model_class(**model_params)
        model.fit(train_df)
        
        # Predict
        preds = model.predict(steps=test_size)
        
        # Evaluate
        metrics = DRPEvaluator.evaluate(
            test_df.values, 
            preds, 
            train_df.values, 
            test_month_num=horizon
        )
        all_metrics.append(metrics)
    
    # Aggregate results
    final_metrics = {}
    for metric in all_metrics[0].keys():
        values = [m[metric] for m in all_metrics if not np.isnan(m[metric])]
        final_metrics[metric] = np.mean(values) if values else np.nan
    
    return final_metrics

# Example usage
if __name__ == "__main__":
    # 加载数据(确保第一列是Part ID，后面是日期列)
    data = pd.read_csv('E:\ZIP-DeepAR代码\data\InterSim层次聚类后的Q料202001-202302(halfmonth).csv', index_col=0)
    #data = pd.read_csv('E:\ZIP-DeepAR代码\data\salestv_data.csv', index_col=0)
    #data = pd.read_csv('E:\ZIP-DeepAR代码\data\carpartsdelete80.csv', index_col=0)
        #将列名转化为202001 ~ 202604月份，避免freq=15d的2-30日期问题
    import calendar
    from datetime import datetime, timedelta

    date_str = '1998-01'
    date_format = '%Y-%m'
    num_months = 76 #51 76 1913

    dates = []
    current_date = datetime.strptime(date_str, date_format)

    for i in range(num_months):
        dates.append(current_date.strftime(date_format))
        num_days = calendar.monthrange(current_date.year, current_date.month)[1]
        current_date += timedelta(days=num_days)

    data.columns = dates
    # Run cross validation
    cv_results = time_series_cross_validate(
        data,
        DRPModel,
        n_splits=3,
        horizon=6,
        lead_time=2,
        safety_factor=1.5,
        alpha=0.1
    )
    
    print("\nCross-validation results:")
    for metric, value in cv_results.items():
        print(f"{metric}: {value:.4f}")
    
    # Full train and evaluation
    train_size = 72
    train_df = data.iloc[:, :train_size]
    test_df = data.iloc[:, train_size:]
    
    model = DRPModel(
        lead_time=2,
        safety_factor=1.5,
        alpha=0.1
    )
    model.fit(train_df)
    
    # Predict
    preds = model.predict(steps=test_df.shape[1])
    
    # Evaluate
    metrics = DRPEvaluator.evaluate(test_df.values, preds, train_df.values)
    
    print("\nTest set evaluation:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    # View inventory plans
    inventory_plans = model.get_inventory_plans()[:3]
    print("\nSample inventory plans:")
    for i, plan in enumerate(inventory_plans):
        print(f"SKU {i}:")
        for k, v in plan.items():
            print(f"  {k}: {v:.2f}")


Cross-validation results:
R2: -0.0920
RMSE: 132.3085
RMSE_non_zero: 205.5562
MAE: 62.0869
MAE_non_zero: 101.8514
MASE: 2.2304
MASE_non_zero: 4.5942
RMSSE: 0.8999
RMSSE_non_zero: 1.5241
QuantileLoss_50: 1.6486
Precision: 0.3515
Recall: 0.0314
F1: 0.0575
MAE_0: 42.5108
RMSE_0: 73.2039
MAE_0_50: 41.9826
RMSE_0_50: 67.8455
MAE_50_100: 73.7444
RMSE_50_100: 101.9926
MAE_100_above: 204.4245
RMSE_100_above: 339.4169

Test set evaluation:
R2: -2.6924
RMSE: 89.2705
RMSE_non_zero: 113.3447
MAE: 52.1896
MAE_non_zero: 74.7085
MASE: 1.1239
MASE_non_zero: 3.0165
RMSSE: 0.5504
RMSSE_non_zero: 1.1822
QuantileLoss_50: 3.3219
Precision: 0.0000
Recall: 0.0000
F1: 0.0000
MAE_0: 45.9879
RMSE_0: 81.3995
MAE_0_50: 46.4944
RMSE_0_50: 79.2846
MAE_50_100: 80.5397
RMSE_50_100: 117.1014
MAE_100_above: 137.8428
RMSE_100_above: 167.7411

Sample inventory plans:
SKU 0:
  reorder_point: 557.60
  safety_stock: 168.27
  lead_time_demand: 389.33
  demand_variance: 6292.46
  forecast: 95.24
SKU 1:
  reorder_point: 1162.8